# Homework:

---

**NAME:**


**Instructions:** 
- The clarity (clean writing) and the organization of the work (numbering the questions appropriately) are taken into account during the correction. 
- Before uploading your notebook, **Kernel/restart and clear output**. And **verify that your code is running cell after cell.**
- This homework is **optional**. It counts at most as 4 **bonus** points on the final grade (over 20) of the course (capped at 20). It must be uploaded back on moodle **BEFORE** the 12th of March at 8PM (Paris time) to be counted, no delay will be accepted. 

---

In [1]:
# Load packages:

# this package allows to work efficiently with arrays
import numpy as np
# this package is used to draw graphs
import matplotlib.pyplot as plt

## Exercise

Consider the matrices 

$$A = \left(\begin{array}{ccc} 1 & 0 & 0 \\ 2 & 3 & 0 \\ 4 & 5 & 6\end{array}\right), \qquad 
  B = \left(\begin{array}{ccc} 1 & 2 & 1 \\ 2 & 5 & 4 \\ 1 & 4 & 6\end{array}\right), $$
and the vector $b = \left(\begin{array}{c} 1 \\ 1 \\ 1\end{array}\right)$.

1) Write down the steps of the forward substitution algorithm to solve the problem $AV = b$. 

2) Write down the steps of the Cholesky algorithm to decompose the matrix $B = L L^T$. 

**Answer**:

1) $V_1 = \frac{b_1}{A_{1,1}} = \frac{1}{1} = 1$ 

$V_2 = \frac{b_2 - A_{2,1}b_1}{A_{2,2}} = \frac{1-2\times 1}{3}= -\frac{1}{3}$

$V_3 = \frac{b_3 - A_{3,1}b_1 - A_{3,2} b_2}{A_{3,3}} = \frac{1-4\times 1-5\times\frac{-1}{3}}{6}= \frac{3 - 12 +5}{3\times6} = -\frac{2}{9}$

Thus $V = (1,-\frac{1}{3},-\frac{2}{9})^T$.

2) 1st step:
$L_{1,1} = \sqrt{B_{1,1}} = 1$
        
$L_{2,1} = \frac{B_{2,1}}{L_{1,1}} = \frac{2}{1} = 2$ 
    
$L_{3,1} = \frac{B_{3,1}}{L_{1,1}} = \frac{1}{1} = 1$
    
2nd step: $L_{2,2} = \sqrt{B_{2,2} - L_{2,1}^2} = \sqrt{5-2^2} = 1$
        
$L_{3,2} = \frac{B_{3,2}-L_{3,1}L_{2,1}}{L_{2,2}} = \frac{4-1\times2}{1} = 2$ 
    
3rd step:  $L_{3,3} = \sqrt{B_{3,3} - L_{3,1}^2 - L_{3,2}^2} = \sqrt{6-2^2-1^2} = 1$

Thus $L = \left(\begin{array}{ccc} 1 & 0 & 0 \\ 2 & 1 & 0 \\ 1 & 2 & 1 \end{array}\right)$

## Problem

**Definition:** A matrix $S$ is orthogonal if it is invertible and its inverse satisfies $S^{-1} = S^T$. 

The goal of this tutorial is to construct an algorithm providing a decomposition of a matrix $A = S U$ into the product of an orthogonal matrix $S$ and an upper triangular one $U$. 

---

### Preliminary computations on orthogonal matrices

1) a) Consider an orthogonal matrix $S$. What values can $det(S)$ have?

b) Are all matrices satisfying this property orthogonal? Prove it or give a counterexample. 

**Answer:** 

a) $det(S) = \pm 1$

b) No, the matrix $Diag(a,a^{-1})$ is not orthogonal. 

2) Prove that the product of orthogonal matrices remains an orthogonal matrix. 

**Answer:** 

$$ (S^1 S^2) (S^1 S^2)^T = S^1 (S^2 (S^2)^T) (S^1)^T = S^1 (S^1)^T = Id$$

3) Prove that the Euclidean norm of a vector is preserved when multiplying it by an orthogonal matrix $S$, i.e. 

$$\|S V\| = \|V\|$$

where the Euclidean norm yields $\|V\| = \sqrt{V^T V}$.

**Answer:** 

$$\| SV \|^2 = (SV)^T (SV) = V^T S^T S V = V^T V = \|V\|^2$$

4) Consider a vector $V \in \mathbb{R}^N$, and define the matrix 

$$ S(V) = Id - 2 \frac{V V^T}{\|V\|^2},$$ 

where $\|V\| = \sqrt{V^T V}$ denotes the Euclidean norm of $V$. 

Prove that $S(V)$ is an orthogonal matrix. 

**Answer:**  

$$ S(V) S(V)^T = \left(Id - 2 \frac{VV^T}{\|V\|^2}\right)\left(Id - 2 \frac{VV^T}{\|V\|^2}\right)^T = \left(Id - 2 \frac{VV^T}{\|V\|^2}\right)\left(Id - 2 \frac{VV^T}{\|V\|^2}\right) = Id - 4\frac{VV^T}{\|V\|^2} + 4\frac{VV^TVV^T}{\|V\|^4} = Id$$

---

### Construction of the algorithm

As in the Gaussian elimination algorithm, we eliminate the subdiagonal coefficients of a matrix $A$. But, instead of using elementary matrices, we use orthogonal matrices $S(V)$ as defined in 4).  

Let us rewrite $A = (C^1, C^2, \dots, C^N)$ as a row vector of column vectors $C^i \in \mathbb{R}^N$. We multiply iteratively the matrix $A$, and therefore each of its column, by an orthogonal matrix $S(V^i)$.  

5) Consider $C^1 \in \mathbb{R}^N$, we seek a vector $V^1 \in \mathbb{R}^N$ such that 

$$S(V^1) C^1 = \left( \begin{array}{c} \alpha \\ 0 \\ \vdots \\ 0 \end{array} \right). \qquad{} (1) $$ 

What are the possible values of $\alpha$? If there are several possibilities, choose one of them here for the rest of the algorithm. 

**Answer:** $\alpha = \pm \|C^1\|$ as it preserves the norm.

6) We seek $V^1\in\mathbb{R}^N$ satisfying (1) and such that $\|V^1\| = 1$. Let us decompose $V^1 = \left( \begin{array}{c} a \\ W \end{array} \right)$ and $C^1 = \left( \begin{array}{c} b \\ X \end{array} \right)$.

a) Prove that $W = kX$ is proportional to $X$. 

b) Write $a$ as a function of $k$ and $X$.

c) Write an equation satisfied by $k$ depending only on $X$ and $b$. 

d) Solve this equation and compute $k$.  

**Answer:** a) One computes

$$S(V^1) C^1 = \left(\begin{array}{c|c} 1-2a^2 & -2aW^T \\ \hline -2aW & Id-2WW^T \end{array}\right)  \left( \begin{array}{c} b \\ X \end{array} \right) = \left(\begin{array}{c} \alpha  \\ 0 \end{array} \right) = \left(\begin{array}{c} b(1-2a^2) - 2aW^T X \\ X-2W(ab+W^TX)  \end{array} \right) $$

Especially, we need $X = 2W(b\sqrt{1-WW^T}+W^TX)$ to be proporitional to $W$. 

b) The normalization gives $a = \sqrt{1-WW^T} = \sqrt{1-k^2 \|X\|^2}$.

c) One has 

$$ X-2W(ab+W^TX) = 0 = X - 2 k X (b\sqrt{1-k^2\|X\|^2}+k\|X\|^2) = X \left[ 1 - 2k (b\sqrt{1-k^2\|X\|^2} + k\|X\|^2)\right] $$

which rewrites for instance

$$  k^2 + k \left(\frac{b}{\|X\|^2}\sqrt{1-k^2\|X\|^2}\right) - \frac{1}{2\|X\|^2} = 0. $$

d) One computes

$$ \left(k^2 - \frac{1}{2\|X\|^2}\right) = - k \left(\frac{b}{\|X\|^2}\sqrt{1-k^2\|X\|^2}\right) \qquad \qquad (2)$$

and 

$$ \left(k^2 - \frac{1}{2\|X\|^2}\right)^2 = k^2 \frac{b^2}{\|X\|^4}(1-k^2\|X\|^2) $$

then 

$$ k^4 \left(1 + \frac{b^2}{\|X\|^2}\right) - k^2 \frac{1}{\|X\|^2} \left(1 + \frac{b^2}{\|X\|^2}\right) + \frac{1}{4\|X\|^4} = 0 $$

and eventually

$$ k^4 - k^2 \frac{1}{\|X\|^2} + \frac{1}{4\|X\|^2\left(\|X\|^2 + b^2\right)} = 0 $$

This provides $k^2 = \frac{1}{2\|X\|^2} \left(1 \pm \frac{b}{\|C^1\|}\right)$. Coming back to (2) fix the sign in this formula, and we obtain eventually  

$$ k = \pm \sqrt{ \frac{1}{2\|X\|^2} \left(1 + \frac{b}{\|C^1\|}\right)}.$$


7) a) What vector operations do you need to perform to compute $S(V^1) C^2$ for some other vector $C^2 \in\mathbb{R}^N$? 

b) How many scalar operations are therefore necessary to compute this product?

c) How many scalar operations are therefore necessary to compute the product $S(V^1) A$, without performing the trivial operations?

**Answer:** 

a) Since $SC = C - V (V^TC)$, we need to perform a scalar product $(V^T C)$, the multiplication of a vector with a scalar, and the sum of two vectors. 

b) 
- Scalar product: $N$ multiplications + $(N-1)$ additions $= 2N-1$ operations
- Multiplication vector-scalar: $N$ multiplications
- Sum of vectors: $N$ additions

Total: $4N-1$ scalar operations
 
c) We perform this to the last $N-1$ columns. For the first one, we only need to do 1 scalar product and a square root for the norm. That is $(4N-1)(N-1) + 2N-1$ operations (or only $(4N-1)(N-1) + 2N$ if we count the root as 1 operation).


8) Now, we need to eliminate the subdiagonal term on the second column without modifying the first column of $A^1 := S(V^1)A$, i.e. we want 

$$S(V^2) A^1 = \left( \begin{array}{c} * \\ \beta \\ 0 \\ \vdots \\ 0 \end{array} \right).$$

For this purpose, we use another matrix $S(V^2)$ satisfying 

$$ S(V^2)_{1,i} = \delta_{1,i}. $$

What does this condition imposes on the entries of the vector $V^2$ ?

**Answer:** $V_1^2 = 0$

9) Again, what is the value of $\beta$ ? And propose a vector $V^2 \in \mathbb{R}^N$ satisfying $\|V^2\| = 1$ and (2).

**Answer:** Writing $S(V^1)C^2 = (*, (\tilde{C}^2)^T)^T$, we simply apply the previous computations to $\tilde{C}^2$.

Then $\beta = \pm\|\tilde{C}^2\|$ and $V^2 = (0, \sqrt{1-WW^T}, W)$ with $W = kX$ and $k = \pm \sqrt{ \frac{1}{2\|X\|^2} \left(1 + \frac{b}{\|\tilde{C}^2\|}\right)}.$

---

In practice, this technique is applied iteratively to all columns until writing a product of the form 

$$\left(\prod\limits_i S(V^i) \right) A = U $$

which provides the decomposition $A = SU$ where $S = \left(\prod\limits_i S(V^i) \right)^T$.